# COVID -19 Exploratory Analysis

## COVID-19_ADT


In [1]:
#start session
import findspark
findspark.init('/opt/cloudera/parcels/SPARK2/lib/spark2')
#from pyspark import SparkContext
#from pyspark.sql import SparkSession, SQLContext
from pyspark.sql import SparkSession
from timeit import default_timer as timer
#from modules import claims_sql
from pyspark.sql.functions import col, round

import sys
import os
import pprint
pp = pprint.PrettyPrinter(indent=4)
#DW Netezza Login Information
import getpass
import json
from datetime import date, datetime
import pandas as pd
pd.set_option('display.max_columns', None,'display.max_columns', None)

from IPython.core.interactiveshell import InteractiveShell #display results setting
InteractiveShell.ast_node_interactivity = "all"

uid = input("Enter LAN ID: ")
pwd = getpass.getpass("Enter LAN pwd: ")

Enter LAN ID: abaner02
Enter LAN pwd: ········


In [2]:
spark = SparkSession \
        .builder \
        .appName("COVID-19_ADT") \
        .config("spark.executor.instances", "5") \
        .config("spark.executor.memory", "48G") \
        .config("spark.executor.cores", "5") \
        .getOrCreate()


In [ ]:
spark.stop()

In [ ]:
sql = '''select * from cln.V_BCBSMA_ADT_CMT_FNL'''
df = spark.sql(sql)

In [ ]:
df.count()

In [ ]:
df.show()

In [ ]:
df_p = df.toPandas()

In [ ]:
df_p.head()

In [ ]:
sql = '''
select distinct 
encounter_diagnosis,
encounter_cheif_complaint,
encounter_diagnosis_icd10,
encounter_type,
encounter_discharge_disposition,
encounter_admitted_inpatient,
encounter_facility_name,
encounter_facility_city,
encounter_facility_zip,
encounter_account_number,
count(distinct member_id)
from cln.V_BCBSMA_ADT_CMT_FNL
where 
--encounter_diagnosis like '*COVID*' or encounter_cheif_complaint like '*COVID*'
encounter_diagnosis like '%COVID%' or encounter_cheif_complaint like '%COVID%' 
or encounter_diagnosis like '%covid%' or encounter_cheif_complaint like '%covid%' 
or encounter_diagnosis like '%CORONAV%' or encounter_cheif_complaint like '%CORONAV%' 
or encounter_diagnosis like '%coronav%' or encounter_cheif_complaint like '%coronav%' 
or encounter_diagnosis like '%CORONA V%' or encounter_cheif_complaint like '%CORONA V%' 
or encounter_diagnosis like '%corona v%' or encounter_cheif_complaint like '%corona v%' 
group by
encounter_diagnosis,
encounter_cheif_complaint,
encounter_diagnosis_icd10,
encounter_type,
encounter_discharge_disposition,
encounter_admitted_inpatient,
encounter_facility_name,
encounter_facility_city,
encounter_facility_zip,
encounter_account_number
'''
df = spark.sql(sql)
df.count()

In [ ]:
df_p1 = df.toPandas()
df_p1

In [ ]:
sql = '''
select distinct 
encounter_diagnosis
from cln.V_BCBSMA_ADT_CMT_FNL

'''
df_diagnosis = spark.sql(sql).toPandas()
df_diagnosis.count()

In [ ]:
sql = '''
select distinct encounter_account_number from from cln.V_BCBSMA_ADT_CMT_FNL

'''
acc = spark.sql(sql).toPandas()
len(acc)

In [4]:
sql = '''
(select * from V_BCBSMA_ADT_CMT_FNL) a'''

#uid = 'abaner02'
df = spark \
        .read \
        .format("jdbc") \
        .option("url", "jdbc:netezza://bntzp01z.bcbsma.com:5480/PDWAPPRP") \
        .option("user", 'abaner02') \
        .option("password", 'Happy2021') \
        .option("driver", "org.netezza.Driver") \
        .option("dbtable", sql) \
        .load()

df.show()

AnalysisException: 'java.lang.RuntimeException: java.io.IOException: Permission denied;'

In [4]:
df.count()

123259

In [5]:
df.createOrReplaceTempView('ADT_DATA')

In [7]:
# load county - zip mapping
df_county_zip = pd.read_csv(os.getcwd()+'/input/county_zip_code_list.csv',sep='|',dtype={'Zip':'str'})
spark.createDataFrame(df_county_zip).createOrReplaceTempView('COUNTY_ZIP')
df_county_zip.head()

,Zip,City,County
0,01001,Agawam,Hampden
1,01002,Amherst,Hampshire
2,01003,Amherst,Hampshire
3,01004,Amherst,Hampshire
4,01005,Barre,Worcester


In [8]:
sql = '''
select a.*,
    SUBSTRING(a.CURR_TIMESTAMP, 1, 10) AS yyyy_mm_dd,
    SUBSTRING(a.CURR_TIMESTAMP, 1, 4) AS yyyy,
    SUBSTRING(a.CURR_TIMESTAMP, 1, 7) AS yyyy_mm,
    SUBSTRING(a.CURR_TIMESTAMP, 6, 5) AS mm_dd,
    b.County as patient_county,
    c.County as facility_county
    from ADT_DATA as a 
        left join COUNTY_ZIP b
            on a.PATIENT_ZIP_CODE = b.Zip
        left join COUNTY_ZIP c
            on a.ENCOUNTER_FACILITY_ZIP = c.Zip
'''
df = spark.sql(sql)
df.createOrReplaceTempView('ADT_DATA_COUNTY')
df.write.mode("overwrite").format('parquet').saveAsTable('analytics.w5_covid_19_ADT_Netezza_'+uid)

In [9]:
df.count()

123259

In [11]:
sql = '''
select * from ADT_DATA_COUNTY
where 
yyyy = '2020' and ENCOUNTER_ACCOUNT_NUMBER = '04025139'
'''
df_tjx = spark.sql(sql)
df_tjx.count()

0

In [ ]:
sql = '''
select 
    yyyy_mm_dd,mm_dd,--facility_county,
    patient_county,count(distinct member_id) as mem_cnt
    from ADT_DATA_COUNTY 
    where yyyy_mm = '2020-03'
    group by yyyy_mm_dd,mm_dd,--facility_county,
    patient_county 
    order by 1 DESC
'''
df_county_grp = spark.sql(sql).toPandas()
df_county_grp.to_csv('county_level_ADT_by_date.csv')
len(df_county_grp) , df_county_grp.head()

In [ ]:
sql = '''
select 
    mm_dd,--facility_county,
    patient_county,count(distinct member_id) as covid_mem_cnt
    from ADT_DATA_COUNTY 
    where yyyy_mm = '2020-03' and
    encounter_diagnosis like '%COVID%' or encounter_cheif_complaint like '%COVID%' 
    or encounter_diagnosis like '%covid%' or encounter_cheif_complaint like '%covid%' 
    or encounter_diagnosis like '%CORONAV%' or encounter_cheif_complaint like '%CORONAV%' 
    or encounter_diagnosis like '%coronav%' or encounter_cheif_complaint like '%coronav%' 
    or encounter_diagnosis like '%CORONA V%' or encounter_cheif_complaint like '%CORONA V%' 
    or encounter_diagnosis like '%corona v%' or encounter_cheif_complaint like '%corona v%' 
    group by mm_dd,--facility_county,
    patient_county 
    order by 1 DESC
'''
df_county_grp_cvid = spark.sql(sql).toPandas()
df_fin = df_county_grp.merge(df_county_grp_cvid, how='left',on=['mm_dd','patient_county']).fillna(0)
df_fin.to_csv('county_level_ADT_by_date_covid.csv')
len(df_fin) , df_fin.head()

In [ ]:
sql = '''
select distinct
    ENCOUNTER_FACILITY_NAME,
    ENCOUNTER_FACILITY_STREET,
    ENCOUNTER_FACILITY_CITY,
    ENCOUNTER_FACILITY_STATE,
    ENCOUNTER_FACILITY_ZIP,
    facility_county,
    yyyy_mm_dd,mm_dd,
    count(distinct member_id) as mem_cnt
from ADT_DATA_COUNTY
   where yyyy = '2020' and ENCOUNTER_FACILITY_STATE = 'MA'
   group by
   ENCOUNTER_FACILITY_NAME,
    ENCOUNTER_FACILITY_STREET,
    ENCOUNTER_FACILITY_CITY,
    ENCOUNTER_FACILITY_STATE,
    ENCOUNTER_FACILITY_ZIP,
    facility_county,
    yyyy_mm_dd,mm_dd
'''
df=spark.sql(sql)
df.write.mode("overwrite").format('parquet').saveAsTable('analytics.w5_covid_19_ADT_Facilities_'+uid)
df.toPandas().to_csv('facilties_county_level_ADT_by_date.csv')
len(df.toPandas()), df.toPandas().head()

In [ ]:
sql = '''
select distinct 
encounter_diagnosis,
encounter_cheif_complaint,
encounter_diagnosis_icd10,
encounter_type,
encounter_discharge_disposition,
encounter_admitted_inpatient,
encounter_facility_name,
encounter_facility_city,
encounter_facility_zip,
encounter_account_number,
count(distinct member_id)
--from cln.V_BCBSMA_ADT_CMT_FNL
from ADT_DATA
where 
--encounter_diagnosis like '*COVID*' or encounter_cheif_complaint like '*COVID*'
encounter_diagnosis like '%COVID%' or encounter_cheif_complaint like '%COVID%' 
or encounter_diagnosis like '%covid%' or encounter_cheif_complaint like '%covid%' 
or encounter_diagnosis like '%CORONAV%' or encounter_cheif_complaint like '%CORONAV%' 
or encounter_diagnosis like '%coronav%' or encounter_cheif_complaint like '%coronav%' 
or encounter_diagnosis like '%CORONA V%' or encounter_cheif_complaint like '%CORONA V%' 
or encounter_diagnosis like '%corona v%' or encounter_cheif_complaint like '%corona v%' 
group by
encounter_diagnosis,
encounter_cheif_complaint,
encounter_diagnosis_icd10,
encounter_type,
encounter_discharge_disposition,
encounter_admitted_inpatient,
encounter_facility_name,
encounter_facility_city,
encounter_facility_zip,
encounter_account_number
'''
df = spark.sql(sql)
df.count()

In [ ]:
df_p1_n = df.toPandas()
df_p1_n

In [ ]:
#Fetch word count for each abstract
#PATIENT_DOB|PATIENT_FULL_ADDRESS|PATIENT_STREET_ADDRESS|PATIENT_CITY|PATIENT_STATE|PATIENT_ZIP_CODE|PATIENT_PHONE_NUMBER|PATIENT_GENDER|          PATIENT_ID|       SUBSCRIBER_ID|PATIENT_DECEASED
#DXCG_RISK_SCORE|PMI_RISK_SCORE|IMPACT_PRO_RISK_SCORE|UNABLE_TO_REACH|LANDMARK_ELIGIBLE|AQC_PROVIDER_IDENTIFIER|CASE_MANAGER|EPISODE_STATUS|ENCOUNTER_DISCHARGE_EVENT|TOP_30_DIAGS|TOP_40_DIAGS|BH_DIAGS|MEMBER_AGE|NEXT_OF_KIN|SECURITY_RISK|GAP_IN_CARE|ED_BOARDING_ALERT|SERVICING_PROVIDER_NPI_NUMBER|BILLING_PROVIDER_NPI_NUMBER|EMERGENT_OR_NON_EMERGENT|DEATH_DATE|FILE_YYYYMMDD| LOAD_DATE|

cols = ['curr_timestamp',
        'member_age','patient_city','patient_state','patient_zip_code','patient_gender','patient_dob',
        'encounter_diagnosis','encounter_cheif_complaint',
        'encounter_diagnosis_icd10','encounter_type','encounter_discharge_disposition','encounter_discharge_event',
        'encounter_admitted_inpatient','encounter_facility_name','encounter_facility_city',
        'encounter_facility_zip','encounter_account_number','episode_status',
        'TOP_30_DIAGS','TOP_40_DIAGS','BH_DIAGS','DXCG_RISK_SCORE','PMI_RISK_SCORE','IMPACT_PRO_RISK_SCORE',
        'SERVICING_PROVIDER_NPI_NUMBER','BILLING_PROVIDER_NPI_NUMBER','LOAD_DATE'
       ]
df_txt= df.select(*cols).toPandas()
df_txt.head()

In [ ]:
df_txt
#https://en.wikipedia.org/wiki/User:Michael_J/County_table

In [ ]:
sql = '''
    (select * from ADMIN.V_BCL_MONTHLY_CALL ) a'''
#uid = 'abaner02'
df = spark \
        .read \
        .format("jdbc") \
        .option("url", "jdbc:netezza://bntzp01z.bcbsma.com:5480/PDWAPPRP") \
        .option("user", uid) \
        .option("password", pwd) \
        .option("driver", "org.netezza.Driver") \
        .option("dbtable", sql) \
        .load()

df.createOrReplaceTempView('BCL_DATA')
df.show()

In [ ]:
sql = '''
select 
    SUBSTRING(a.INTERACTION_DATE, 1, 10) AS yyyy_mm_dd,
        SUBSTRING(a.INTERACTION_DATE, 1, 4) AS yyyy,
        SUBSTRING(a.INTERACTION_DATE, 1, 7) AS yyyy_mm,
        SUBSTRING(a.INTERACTION_DATE, 6, 5) AS mm_dd,
        count(distinct MEM_NUM) as mem_cnt
        
from BCL_DATA a
--where SUBSTRING(a.INTERACTION_DATE, 1, 4)  = '2020'
group by
    SUBSTRING(a.INTERACTION_DATE, 1, 10),
        SUBSTRING(a.INTERACTION_DATE, 1, 4),
        SUBSTRING(a.INTERACTION_DATE, 1, 7),
        SUBSTRING(a.INTERACTION_DATE, 6, 5)

'''
df = spark.sql(sql).toPandas()
len(df),df.head()

In [ ]:
print(len(df[df['yyyy'] == '2017']), sum(df[df['yyyy'] == '2017']['mem_cnt']))
print(len(df[df['yyyy'] == '2018']), sum(df[df['yyyy'] == '2018']['mem_cnt']))
print(len(df[df['yyyy'] == '2019']), sum(df[df['yyyy'] == '2019']['mem_cnt']))
df[df['yyyy'] == '2019'].head()